In [1]:
import requests
import random
import time
import os

# --- Configurações ---
BASE_URL = "http://localhost:8000/api/v1"
# Garante que o script encontre o PDF, não importa de onde ele seja executado
PROJECT_ROOT = os.getcwd()
PDF_PATH = os.path.join(PROJECT_ROOT, "edital_seap.pdf")

# Dados do usuário de teste (usaremos um e-mail aleatório para cada execução)
USER_EMAIL = f"testuser_{int(time.time())}@example.com"
USER_PASSWORD = "a_strong_password"
USER_NAME = "Test User"

user_payload = {"name": USER_NAME, "email": USER_EMAIL, "password": USER_PASSWORD}
response = requests.post(f"{BASE_URL}", json=user_payload)
user_id = response.json()["id"]
print(f"✅ Sucesso! Usuário criado com ID: {user_id}")

✅ Sucesso! Usuário criado com ID: 1


In [2]:
login_payload = {"username": USER_EMAIL, "password": USER_PASSWORD}
response = requests.post(f"{BASE_URL}/token", data=login_payload)
access_token = response.json()["access_token"]
headers = {"Authorization": f"Bearer {access_token}"}
print("✅ Sucesso! Token gerado.")

✅ Sucesso! Token gerado.


In [10]:
# ENVIA O EDITAL
with open(PDF_PATH, "rb") as f:
    files = {"file": (os.path.basename(PDF_PATH), f, "application/pdf")}
    response = requests.post(f"{BASE_URL}/contests/upload", files=files, headers=headers)

contest_id = response.json()["id"]
print(f"✅ Sucesso! Upload recebido. ID do concurso: {contest_id}. Aguardando processamento...")

✅ Sucesso! Upload recebido. ID do concurso: 7. Aguardando processamento...


In [11]:
# 4. FAZER INSCRIÇÃO EM UM CARGO
# ===============================
print(f"\n[4/6] 🎓 Inscrevendo-se em um cargo...")
response = requests.get(f"{BASE_URL}/contests/", headers=headers)

available_contests = response.json()

# Pega o primeiro cargo do primeiro concurso disponível
role_id = available_contests[0]["roles"][0]["id"]
response = requests.post(f"{BASE_URL}/study/subscribe/{role_id}", headers=headers)

user_contest_id = response.json()["id"]
print(f"✅ Sucesso! Inscrição realizada. ID da inscrição: {user_contest_id}")


[4/6] 🎓 Inscrevendo-se em um cargo...
✅ Sucesso! Inscrição realizada. ID da inscrição: 1


In [12]:
# 5. ENVIAR AUTOAVALIAÇÃO
# =========================
print(f"\n[5/6] 📊 Enviando autoavaliação de proficiência...")
response = requests.get(f"{BASE_URL}/study/user-contests/{user_contest_id}/subjects", headers=headers)
    
subjects = response.json()
proficiency_payload = {
    "proficiencies": [
        {"subject": subject, "score": round(random.uniform(0.1, 0.9), 2)} for subject in subjects
    ]
}

response = requests.post(
    f"{BASE_URL}/study/user-contests/{user_contest_id}/proficiency",
    json=proficiency_payload,
    headers=headers
)

print("✅ Sucesso! Proficiência enviada.")


[5/6] 📊 Enviando autoavaliação de proficiência...
✅ Sucesso! Proficiência enviada.


In [13]:
# 6. CRIAR PLANO DE ESTUDO
# =========================
print(f"\n[6/6] 🧠 Gerando plano de estudo com a IA... (Isso pode demorar)")
response = requests.post(
    f"{BASE_URL}/study/user-contests/{user_contest_id}/generate-plan",
    headers=headers,
    timeout=300 # Timeout de 5 minutos para a IA
)
    
result = response.json()
items_created = result.get("roadmap_items_created", 0)
print(f"✅ SUCESSO TOTAL! Plano de estudo gerado com {items_created} sessões.")
print("🎉 Pipeline concluído com sucesso!")


[6/6] 🧠 Gerando plano de estudo com a IA... (Isso pode demorar)
✅ SUCESSO TOTAL! Plano de estudo gerado com 84 sessões.
🎉 Pipeline concluído com sucesso!
